# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
## Get relevant data!

# test if your_file does not (!) exist (-e), and import if true
!test ! -e deep-learning-v2-pytorch && git clone https://github.com/udacity/deep-learning-v2-pytorch.git

# move the .py files to pwd
!mv deep-learning-v2-pytorch/project-tv-script-generation/*.py /content
!mv deep-learning-v2-pytorch/project-tv-script-generation/data/ /content

# remove deep learning folder
!rm -r deep-learning-v2-pytorch

mv: cannot stat 'deep-learning-v2-pytorch/project-tv-script-generation/*.py': No such file or directory
mv: cannot stat 'deep-learning-v2-pytorch/project-tv-script-generation/data/': No such file or directory


In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [3]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [4]:
text[:100]

'jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and '

In [5]:
words = text.split(' ')
print(len(words))
print(words[:10])

605704
['jerry:', 'do', 'you', 'know', 'what', 'this', 'is', 'all', 'about?', 'do']


In [6]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    words = set(text)
    int_to_vocab = dict(enumerate(words))
    vocab_to_int = {str(word): n for n, word in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [7]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    token_dict = {
        ".": "||Exclamation_Mark||",
        ",": "||Comma||",
        '"': '||Quotation_Mark||',
        ";": "||Semicolon||",
        "!": "||Exclamation_Mark||",
        "?": "||Question_Mark||",
        "(": "||Left_Parentheses||",
        ")": "||Right_Parentheses||",
        "-": "||Dash||",
        "\n": "||Return||"
    }
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [10]:
print(len(int_text))
print(len(vocab_to_int))
print(len(int_to_vocab))
print(len(set(int_text)))

892110
21387
21387
21386


In [11]:
print(min(int_to_vocab.items()), max(int_to_vocab.items()))
print(min(vocab_to_int.items()), max(vocab_to_int.items()))
print(vocab_to_int['#'], int_to_vocab[859])

(0, 'khomeini') (21386, 'defrost')
('#', 11436) ('\x93sari\x94', 9771)
11436 madeline


In [12]:
for i in int_text:
    if i == 5759:
        print(i)

5759


In [13]:
vocab_to_int['elaine']

9004

In [14]:
int_to_vocab[5759]

'limits'

In [127]:
print(len(words))
print(len(text))

605704
3471464


In [128]:
sample = int_text[:10]
print(sample)

[17589, 13790, 79, 5303, 5303, 5303, 7363, 79, 13790, 9936]


In [17]:
for s in sample:
    print(int_to_vocab[s])

this
is
out
||exclamation_mark||
||exclamation_mark||
||exclamation_mark||
and
out
is
one


## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    device = 'cpu'
    print('No GPU found. Please use a GPU to train your neural network.')
else:
    device = 'cuda'
print(device)

cuda


In [0]:
# This is supposedly to help prevent some cuda errors but I didn't notice a difference. 
# I think my errors were vocab-related.
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [0]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    n_batches = len(words)//(sequence_length*batch_size)
#     print('n_batches', n_batches)
    n_targets = words[:-sequence_length]
#     print('len words', len(words))
    
    x, y = [], []
    for i in range(len(n_targets)):
        batch_x = words[i:i+sequence_length]
        try:
            batch_y = words[i+sequence_length]
        except IndexError:
            batch_y = words[0]
        x.append(batch_x)
        y.append(batch_y)
    
    x, y = np.array(x), np.array(y)
#     print('x types:', type(x), x.shape, 'len(y):', len(y))
    x, y = torch.from_numpy(x), torch.from_numpy(y)
    
    data = TensorDataset(x, y)
    
    # return a dataloader
    return DataLoader(data, batch_size=batch_size, shuffle=False)

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own
# With great help from https://knowledge.udacity.com/questions/29798

In [22]:
ex = list(range(101))
exx = batch_data(ex, 5, 10)
print('\nexx:', exx)


exx: <torch.utils.data.dataloader.DataLoader object at 0x7f25ec146048>


In [23]:
x = iter(exx)
x

In [24]:
next(x)

[tensor([[ 0,  1,  2,  3,  4],
         [ 1,  2,  3,  4,  5],
         [ 2,  3,  4,  5,  6],
         [ 3,  4,  5,  6,  7],
         [ 4,  5,  6,  7,  8],
         [ 5,  6,  7,  8,  9],
         [ 6,  7,  8,  9, 10],
         [ 7,  8,  9, 10, 11],
         [ 8,  9, 10, 11, 12],
         [ 9, 10, 11, 12, 13]]),
 tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])]

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [25]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]])

torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the LSTM to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [26]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        # set class variables
#         print('RNN initted: \nvocab_size {}, output_size {}, \nembedding_dim {}, hidden_dim {} \nn_layers {}'.format(vocab_size, output_size, embedding_dim, hidden_dim, n_layers))
        self.vocab_size = vocab_size
        self.output_size = output_size
        self.embedding_dim = embedding_dim
        self.drop_prob = dropout
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        # define model layers
        self.embed = nn.Embedding(vocab_size, embedding_dim)
#         self.dropout = nn.Dropout(self.drop_prob)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=self.drop_prob, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)  
#         self.activation = nn.Sigmoid()
#         self.activation = nn.LogSoftmax(dim=0)
#         print('\nlstm model:', self)
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        
#         print('\nForward parameter shapes: \nnn_input: {}, \nhidden: {}, {}'.format(nn_input.shape, hidden[0].shape, hidden[1].shape))
        batch_size = nn_input.shape[0]
        seq_length = nn_input.shape[1]
#         print('So batch_size is {} | sequence length is {}'.format(batch_size, seq_length))
#         print()

        embeds_out = self.embed(nn_input) # out shape: [N, M, embedding_dim]
#         print('\nembeds_out == [N ({}), M ({}), embedding_dim ({})] --> {}'.format(batch_size, seq_length, self.embedding_dim, embeds_out.shape))
        
        lstm_out, hidden = self.lstm(embeds_out, hidden) # out shape: [N, M, hidden_dim]
#         print('\nlstm_out == [N ({}), M ({}), hidden_dim ({})] --> {}'.format(batch_size, seq_length, self.hidden_dim, lstm_out.shape))
#         lstm_out = self.dropout(lstm_out)
        
        # flatten for linear layer
        fc_out = self.fc(lstm_out.contiguous().view(-1, self.hidden_dim)) # out shape: [N*M, output_size]
#         print('fc_out shape == [N*M ({}), output_size ({})] --> {}'.format((batch_size*seq_length), self.output_size, fc_out.shape))
#         sigm_out = self.activation(fc_out) # out shape: [N*M, output_size]
        
        # reshape back to original shape
        out = fc_out.view(batch_size, -1, self.output_size) # out shape: [N, M, output_size]
#         print('out == [N, M, output_size] -->', out.shape)
#         print('out[:5]', out[:5])
        final = out[:, -1]
#         print('final == [:, -1] -->', final.shape)
#         print('final[:5]\n', final[:5])

        # return one batch of output word scores and the hidden state
        return final, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight = next(self.parameters()).data
        
        # initialize hidden state with zero weights, and move to GPU if available
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
#         print('\nhidden initted:', hidden[0].shape, hidden[1].shape, '\n', hidden[0][0, :2], '\n', hidden[1][0, :2])
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [27]:
if train_on_gpu:
    print(device)

cuda


In [28]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    # move data to GPU, if available
    if (train_on_gpu):
        inputs, targets = inp.cuda(), target.cuda()
    else:
        inputs, targets = inp, target

#     print('\nInputs:', inputs.dtype, inputs.shape)
#     print('Targets:', targets.dtype, targets.shape)
#     print('Hidden[0] before (s/b all zeros?):', hidden[0].dtype, hidden[0].shape)
    
    batch_size, sequence_length = inputs.shape
    
    ### Within the batch loop, we detach the hidden state from its history; 
    ### this time setting it equal to a new tuple variable because an LSTM 
    ### has a hidden state that is a tuple of the hidden and cell states.
    hidden = tuple([each.data for each in hidden])
    
    ### zero-grad the rnn - people say to do this here, on model, to make sure it
    rnn.zero_grad()
#     optimizer.zero_grad()
    
#     print('Hidden[0] after detached:', hidden[0].dtype, hidden[0].shape)
    output, hidden = rnn(inputs, hidden) # the forward function?
#     print('Hidden[0] after forward:', hidden[0].dtype, hidden[0].shape)
#     print('\nps:', output.dtype, output.shape)
#     print('targets:', targets.dtype, targets.shape)
    
#     # reshape targets
#     print('\nbatch_size {}, seq_len {}'.format(batch_size, sequence_length))
# #     targets = targets.view(batch_size*sequence_length).long()
#     print('\noutput:', output.dtype, output.shape)
#     print('\noutput after squeeze:', output.squeeze().dtype, output.squeeze().shape)
#     print('targets:', targets.dtype, targets.shape)
    
#     print('\ntargets:\n', targets)
#     print('\noutput:\n', output)

    ### Remember in the Forward function, we smooshed the rnn outputs into (batch_size * seq_length).
    ### Do the same for the targets!!!
#     targets.view(batch_size * sequence_length)

    loss = criterion(output.squeeze(), targets) # All models and loss functions assume 
                                            # that the first dimension is the batch size. 
                                            # So the first input to cross entropy should be 
                                            # batch_size x nb_classes and the second batch_size 
                                            # (and contain values between 0 and nb_classes-1).
    
    # perform backpropagation and optimization
   
    loss.backward()
    
    ### clip the gradients so they don't explode!
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    
#     print('\nave_loss: {}, hidden: {}'.format(ave_loss, hidden[0].shape))
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
#         print('About to init_hidden --> batch_size is', batch_size)
        hidden = rnn.init_hidden(batch_size)
#         print('rnn parameters:', rnn)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [0]:
# Data params
# Sequence Length -- length of the memory of word order
sequence_length =   8 # of words in a sequence
# Batch Size -- powers of 2
batch_size = 256

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [0]:
# Training parameters
# Number of Epochs
num_epochs = 20
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size -- all the words in our dictionary
vocab_size = len(int_to_vocab)
# Output size -- # classes, ie len of our dictionary
output_size = vocab_size
# Embedding Dimension  -- can be between about 200-400
embedding_dim = 200
# Hidden Dimension -- between about 128-512, in powers of 2
hidden_dim = 128
# Number of RNN Layers
n_layers = 1

# Show stats for every n number of batches
show_every_n_batches = 1500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [49]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [122]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./gdrive/My Drive/trained_rnn_3layers', trained_rnn)
print('Model Trained and Saved')

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Training for 20 epoch(s)...
Epoch:    1/20    Loss: 4.795968067646027

Epoch:    1/20    Loss: 4.286972825050354

Epoch:    2/20    Loss: 4.018076959877245

Epoch:    2/20    Loss: 3.8580244369506835

Epoch:    3/20    Loss: 3.7622163102271093

Epoch:    3/20    Loss: 3.6537115009625754

Epoch:    4/20    Loss: 3.5932151002268635

Epoch:    4/20    Loss: 3.509102011203766

Epoch:    5/20    Loss: 3.4641427487855956

Epoch:    5/20    Loss: 3.3957300391197203

Epoch:    6/20    Loss: 3.3575296440432147

Epoch:    6/20    Loss: 3.298321128845215

Epoch:    7/20    Loss: 3.2661419015738273

Epoch:    7/20    Loss: 3.2138905634880066

Epoch:    8/20    Loss: 3.1861682993269738

Epoch:    8/20    Loss: 3.139856357415517

Epoch:    9/20    Loss: 3.115555876685727

Epoch:    9/20    Loss: 3.071970047632853

Epoch:   10/20    Loss: 3.0527827796195783

Epoch:   10/20    Loss: 3.01268159532547

Epoch:   11/20    Loss: 2.9965366959091155

Epoch:   11/20    Loss: 2.9578192496299742

Epoch:   12/20

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** To be honest, mostly it was random choices. I started with hyperparameters that were similar to the previous projects and went from there - something like:
- batch_size = 64
- hidden_dim=512
- epochs=50
- output_size=1

The output_size was obviously giving me errors and someone steered me in the right direction. Since a higher hidden_dim and more layers will generally make a model train better, ideally you want those as high as possible. As well, a higher batch_size should train faster. However, they're all at the expense of speed.

I have a bad habit of changing more than 1 parameter at a time, sometimes, and not keeping a log (I'd make a bad scientist!) so I'm not too sure how I ended up with these final parameters. But I was egged on by fellow Spartans (classmates) who said they could get great results in 10-15 epochs, so I kept adjusting. 

It's probably a coincidence, but every time I trained at sequence_length=8, my model did converge faster.

Hey look at that! I'm writing this as I'm training my final attempt and it's the results I had been trying for all day! This time, I think I lowered sequence_length from 10 to 8, lowered n_layers from 2 to 1 and lowered the hidden_dim from 256 to 128. My previous results had been about 3.35 in 20 epochs so this is much faster! 

Of course, I'd like to keep adjusting to get a better/faster result, but I'd like to submit this tonight!  :)

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('content/gdrive/My Drive/trained_rnn.pt')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
#         current_seq = np.roll(current_seq, -1, 1)
        current_seq = np.roll(current_seq.cpu(), -1, 1) # edited to prevent error
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [125]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'elaine' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

elaine: you know what the way are you?

elaine: well, it's the same thing i want to be able to get this up to a while.

george: hey.

elaine: what? why?

george: no. i got a big favor.

george: what?

george: it's right, and it's a good idea.

jerry: what?

kramer: oh, well i am.

george:(jokingly mimicking the door) oh, i think we should go.

elaine:(pointing at davids) yeah, yeah..

kramer: oh, come on.

jerry:(pointing to george) you know what i think you can do. it's a lot of passion.

jerry: i don't know what you want about you a little bit?

george: oh, yeah. i don't know how much to be the best way in there.

george: i think it's a lot of people in the same day.

elaine: hey, hey, what are we gonna do?

george: no, no. i'm not going to be a good idea, it's a good thing.

jerry: what are you gonna say about?

jerry: yeah, i can't get a good.

kramer: yeah, i got to be honest. you know what the hell i was. you know, i have a lot of furs and you know what the way is that?

jerry: w

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [0]:
# save script to a text file
f =  open("gdrive/My Drive/PyTorch!/DLND/tvscripts/generated_script_final.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.